# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [68]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from citipy import citipy


# Import API key
from api_keys import g_key




### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [69]:
df = pd.read_csv('weather_data.csv')
df = pd.DataFrame(df)


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [70]:
df = df.dropna(subset = ['lat','lon'])
gmaps.configure(api_key=g_key)
locations = df[["lat", "lon"]].astype(float)
humidity = df["humidity"].astype(float)

In [71]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [72]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.

is_hot = df['temp_max'] >= 293
df1 = df[is_hot]
is_comfortable = df['temp_max'] <= 300
df1 = df1[is_comfortable]
breezy = df1['wind_speed'] <= 10
df1 = df1[breezy]
clear_sky = df1['cloudiness'] <=10
df1 = df1[clear_sky]

df1.head()


C:\Users\brian\Anaconda4\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,coord,weather,base,main,visibility,wind,clouds,dt,sys,timezone,...,snow,lat,lon,temp,temp_min,temp_max,cloudiness,wind_speed,wind_direction,humidity
98,"{'lon': 167.48, 'lat': -22.67}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 299.27, 'feels_like': 299.26, 'temp_m...",NaN,"{'speed': 6.39, 'deg': 100}",{'all': 8},1.582431e+09,"{'country': 'NC', 'sunrise': 1582396972, 'suns...",39600.0,...,NaN,-22.67,167.48,299.27,299.27,299.27,8.0,6.39,100.0,76.0
165,"{'lon': -155.09, 'lat': 19.73}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 293.9, 'feels_like': 290.68, 'temp_mi...",16093.0,"{'speed': 5.1, 'deg': 120}",{'all': 1},1.582431e+09,"{'type': 1, 'id': 7881, 'country': 'US', 'sunr...",-36000.0,...,NaN,19.73,-155.09,293.90,285.93,299.15,1.0,5.10,120.0,54.0
179,"{'lon': 84.6, 'lat': 18.93}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 297.18, 'feels_like': 296.64, 'temp_m...",NaN,"{'speed': 0.26, 'deg': 91}",{'all': 0},1.582431e+09,"{'country': 'IN', 'sunrise': 1582418742, 'suns...",19800.0,...,NaN,18.93,84.60,297.18,297.18,297.18,0.0,0.26,91.0,37.0
200,"{'lon': -50.21, 'lat': -30.18}","[{'id': 500, 'main': 'Rain', 'description': 'l...",stations,"{'temp': 294.33, 'feels_like': 293.54, 'temp_m...",NaN,"{'speed': 2.98, 'deg': 200}",{'all': 3},1.582431e+09,"{'country': 'BR', 'sunrise': 1582448822, 'suns...",-10800.0,...,NaN,-30.18,-50.21,294.33,294.33,294.33,3.0,2.98,200.0,64.0
214,"{'lon': -67.58, 'lat': -39.03}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 295.15, 'feels_like': 291.59, 'temp_m...",10000.0,"{'speed': 3.1, 'deg': 70}",{'all': 0},1.582431e+09,"{'type': 1, 'id': 8310, 'country': 'AR', 'sunr...",-10800.0,...,NaN,-39.03,-67.58,295.15,295.15,295.15,0.0,3.10,70.0,30.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [73]:
hotel_df = pd.DataFrame(df1, columns =['name','lat', 'lon','wind','Country'])
hotel_df['Hotel Name']=''
hotel_df = hotel_df.rename(columns = {'name':'City'})
hotel_df

,City,lat,lon,wind,Country,Hotel Name
98,Vao,-22.67,167.48,"{'speed': 6.39, 'deg': 100}",NaN,
165,Hilo,19.73,-155.09,"{'speed': 5.1, 'deg': 120}",NaN,
179,Sompeta,18.93,84.60,"{'speed': 0.26, 'deg': 91}",NaN,
200,Cidreira,-30.18,-50.21,"{'speed': 2.98, 'deg': 200}",NaN,
214,General Roca,-39.03,-67.58,"{'speed': 3.1, 'deg': 70}",NaN,
248,Abu Dhabi,24.47,54.37,"{'speed': 1.5, 'deg': 100}",NaN,
250,Kabala,9.58,-11.55,"{'speed': 1.96, 'deg': 215}",NaN,
269,Bhadrachalam,17.67,80.88,"{'speed': 1.54, 'deg': 134}",NaN,
281,Jacmel,18.23,-72.53,"{'speed': 1.5, 'deg': 260}",NaN,
295,Sur,22.57,59.53,"{'speed': 1.2, 'deg': 174}",NaN,


In [74]:
hotel = []
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
cities = hotel_df['City']
parameters = { "rankby": "distance",
    "type": "hotel",
    "key": g_key,}

for index, row in df1.iterrows():
     # get restaurant type from d
    lat = row["lat"]
    long = row["lon"]
    parameters["location"] = f"{lat}, {long}"

    # assemble url and make API request
    response = requests.get(url, params=parameters)
    
    # convert to json
    response = response.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
        hotel.append(response['results'][0]['name'])
    except IndexError:
        hotel.append(np.nan)



In [75]:
hotel_df['hotel'] = hotel
hotel_df = hotel_df.dropna()
hotel_df.to_csv('hotels.csv')

In [77]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [79]:
# Add marker layer ontop of heat map
layer = gmaps.symbol_layer(locations,info_box_content=hotel_df)
fig.add_layer(layer)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))